In [1]:
import json
from datasets import load_dataset

In [2]:
# Load the dataset
dataset = load_dataset("potsawee/speecheval-advanced-v1")["train"]
print("len(dataset):", len(dataset))

len(dataset): 82


In [5]:
def read_judge_output_dir(judge_output_dir):
    winner_ref, winner_can = 0, 0
    for i in range(len(dataset)):
        x = dataset[i]
        conversation_id = x["id"]
        ref_model_type = x["position"]
        judge_output_path = f"{judge_output_dir}/{conversation_id}.txt"
        with open(judge_output_path) as f:
            y = json.load(f)
        response = y['response']
        # calculate winner ratio
        verdict = response.split("[Rankings]")[-1].strip("```").strip("json").strip("python").strip()
        parsed = json.loads(verdict)
        yy = parsed['rankings'][0]
    
        if int(yy['rank']) == 1:
            if yy['model'] == 'model_1' and ref_model_type == 'model_1':
                winner_ref += 1
            elif yy['model'] == 'model_1' and ref_model_type == 'model_2':
                winner_can += 1
            elif yy['model'] == 'model_2' and ref_model_type == 'model_1':
                winner_can += 1
            elif yy['model'] == 'model_2' and ref_model_type == 'model_2':
                winner_ref += 1
            else:
                raise Exception()
        else:
            raise Exception()
    print("candidate winner percentage: {:.2f}%".format(winner_can / (winner_can + winner_ref) * 100))
    print("total:", winner_can + winner_ref)

In [9]:
read_judge_output_dir("./judge_outputs/advvoiceq1_judge_gpt4o_candidate_gpt4o/")

candidate winner percentage: 47.56%
total: 82


In [7]:
read_judge_output_dir("./judge_outputs/advvoiceq1_judge_gpt4o_candidate_4okokoro/")

candidate winner percentage: 29.27%
total: 82


In [10]:
read_judge_output_dir("./judge_outputs/advvoiceq1_judge_gpt4o_candidate_typhoon2/")

candidate winner percentage: 17.07%
total: 82


In [12]:
read_judge_output_dir("./judge_outputs/advvoiceq1_judge_gpt4o_candidate_typ2kokoro/")

candidate winner percentage: 12.20%
total: 82
